In [2]:
import nltk
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize 
#nltk.download('wordnet')
#nltk.download('stopwords')
from nltk.corpus import wordnet #wornet loaded
import re
import json
from flask import Flask,render_template, request, redirect, url_for
from flask import jsonify, current_app
from flask_cors import CORS

#Methode definition
# https://stackoverflow.com/questions/1883980/find-the-nth-occurrence-of-substring-in-a-string
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    print("le debut",start,n)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    print(start)
    return start

def get_synsets(w):
    syns = wordnet.synsets(w)
    return syns

def get_definitions(syns):
    return syns.definition()

def get_label(syns):
    lemmas = syns.lemmas()
    name = lemmas[0].name()
    return name

def get_class(syns):
    return syns.pos()

def get_resource(syns):
    return syns.offset()  #id for a synset in wordnet db

def get_source(syns):
    return syns.name()

#TODO what if the text is used multiple times?
def get_offset(w,text,nth):
    offset = find_nth(text, w, nth)
    return offset

def stop_words_filtering(text):
    reg_exp = r"[a-zA-Z0-9_-]+"  #Reguläre Ausdruck, der alle erlaubten Sylabeln enthält. Von "a" bis "z"
                            #von "A" bis "Z"   
    #reg_exp2 = r"[a-zA-Z0-9]+"
    stop_words  = set(stopwords.words('english')) #stopwörter für englisches Text
    word_tokens = []
    #word_token2 = []
    a = re.compile(reg_exp)
    #b = re.compile(reg_exp2)
    word_tokens = word_tokens + a.findall(text) #löscht die Satzzeichnen im Text
    filtered_sentence = filtered_sentence2 =  [w for w in word_tokens if not w in stop_words] #löscht die Stopp-Wörter im Text
    #word_tokens2 = word_tokens2 + b.findall(text) #für die wörter wie pet_food pet-food (-,_ löschen)
     
    return filtered_sentence 
#filtered_sentence = [] 
    #for w in word_tokens: 
      #  if w not in stop_words: 
    #        filtered_sentence.append(w)

#Nested dictionary
def build_ressources_candidates(word,syns,offset):
    resourcedict = {}
    resourcedict["description"] = get_definitions(syns)
    resourcedict["label"]       = get_label(syns)
    resourcedict["offset"]      = offset
    resourcedict["resource"]    = get_resource(syns)
    resourcedict["source"]      = get_source(syns)
    resourcedict["text"]        = word
    resourcedict["pos"]         = get_class(syns) #pos(part of speech oder word class)
    return resourcedict   
    
def build_resources(word,offset):
    resources = []
    synsets = get_synsets(word)
    if "-" in word:
        tokens = word.split("-")
        for w in tokens:
            synsets.extend(get_synsets(w))
    elif "_" in word:
        tokens = word.split("_")
        for w in tokens:
            synsets.extend(get_synsets(w))    
    for syns in synsets:
        resources.append(build_ressources_candidates(word,syns,offset))
    return resources

def build_annot_candidates(word,text,current_word_occurence_count):
        offset = get_offset(word,text,current_word_occurence_count)
        candidate = {}
        candidate["offset"]                = offset
        candidate["resource_candidates"]  = build_resources(word,offset)
        candidate["text"]                  = word
        return candidate
def build_annot(text):
   
    annotDict = {}
    annot = []
    annotDict["annotation_candidates"]   = annot
    annotDict["text"]                    = text
    Woerter = stop_words_filtering(text)
    wordCount = {}
    print('currentWords:',Woerter)
    #print(Woerter)
    for w in Woerter:
        find = 0
        if not bool(wordCount): #wenn dictionary leer ist
            wordCount[w] = 0
            wordCount[w] = wordCount[w] + 1
            currentWordCount = wordCount[w]
            print("Start:",w)
        else:
            #if not (w in wordCount.keys()):
            #wordCount[w] = 0
            for key in wordCount.copy():
                #if (key.find(w) == -1):   
                if (key.find(w) != -1):
                    print("ich bin da find ist true:", key,w)
                    wordCount[key] = wordCount[key] + 1
                    currentWordCount = wordCount[key]
                    find = 1
                   # print("after",wordCount[w])
                    break
            #print("ich bin da:", key,w)
            if find != 1:
                print("not find: ", w)
                wordCount[w] = 0
                wordCount[w] = wordCount[w] + 1
                currentWordCount = wordCount[w]
                   # print("actual",currentWordCount)
                #print("nach",wordCount[w])
        annot.append(build_annot_candidates(w,text, currentWordCount))
                    
        #if not (w in wordCount.keys()):
            #wordCount[w] = 0
        
            #annot.append(build_annot_candidates(w,text, currentWordCount))
    print(wordCount)
    return annotDict

    
    
    

In [3]:
#json.dumps(build_annot("say that you say"))
#import nltk
#nltk.download('stopwords')
#nltk.download('wordnet')
#d = stop_words_filtering("I am a winners, but not a looser 9!")
#print(d)
#import sys
#sys.stdout = open('synset.png', 'w')
#print(get_synsets('Framework'))

  # print()
#annotation = build_annot("pet-food,petfood and pet")
#dic = {"pet":"food"}
#text = "pet-food"
#separator = ''
#print(bool(dic))
#print(text.startswith("pet"))
#print(text.find("food"))
#print(text.index("food"))
#print(enumerate(text))
#for index, word in enumerate(nltk.word_tokenize(text)):
#    print(index, word)
#print(annotation)


In [ ]:
app = Flask(__name__)
CORS(app)
@app.route('/')
def index():
  return 'Server Works done!'
    
@app.route('/annotApi/<string:text>')
def annotsFunction2(text):
    annotation = build_annot(text)
    #print(json.dumps(annotation))
    return json.dumps(annotation)

    
if __name__ == '__main__':
    with app.app_context():
   # app.debug = True
        from werkzeug.serving import run_simple
        run_simple('localhost', 4000, app)

 * Running on http://localhost:4000/ (Press CTRL+C to quit)


currentWords: ['This', 'technology', 'used', 'explore', 'map', 'deeper', 'parts', 'ocean', 'visibility', 'either', 'low', 'non-existent', 'Hooking', 'device', 'machine', 'withstand', 'deep', 'depths', 'ocean', 'explore', 'parts', 'ocean', 'may', 'able']
Start: This
le debut 0 1
0


127.0.0.1 - - [17/Jan/2020 02:18:55] "GET /annotApi/This%20technology%20can%20be%20used%20to%20explore%20and%20map%20deeper%20parts%20of%20the%20ocean%20where%20visibility%20is%20either%20low%20or%20non-existent.%20Hooking%20up%20this%20device%20to%20a%20machine%20that%20can%20withstand%20the%20deep%20depths%20of%20the%20ocean,%20we%20can%20now%20explore%20parts%20of%20the%20ocean%20that%20we%20may%20not%20have%20been%20able%20to%20do%20before. HTTP/1.1" 200 -


not find:  technology
le debut 5 1
5
not find:  used
le debut 23 1
23
not find:  explore
le debut 31 1
31
not find:  map
le debut 43 1
43
not find:  deeper
le debut 47 1
47
not find:  parts
le debut 54 1
54
not find:  ocean
le debut 67 1
67
not find:  visibility
le debut 79 1
79
not find:  either
le debut 93 1
93
not find:  low
le debut 100 1
100
not find:  non-existent
le debut 107 1
107
not find:  Hooking
le debut 121 1
121
not find:  device
le debut 137 1
137
not find:  machine
le debut 149 1
149
not find:  withstand
le debut 166 1
166
ich bin da find ist true: deeper deep
le debut 47 2
180
not find:  depths
le debut 185 1
185
ich bin da find ist true: ocean ocean
le debut 67 2
199
ich bin da find ist true: explore explore
le debut 31 2
217
ich bin da find ist true: parts parts
le debut 54 2
225
ich bin da find ist true: ocean ocean
le debut 67 3
238
not find:  may
le debut 252 1
252
not find:  able
le debut 270 1
270
{'This': 1, 'technology': 1, 'used': 1, 'explore': 2, 'map': 1, '

127.0.0.1 - - [17/Jan/2020 02:19:15] "GET /annotApi/This%20technology%20can%20be%20used%20to%20explore%20and%20map%20deeper%20parts%20of%20the%20ocean%20where%20visibility%20is%20either%20low%20or%20non-existent.%20Hooking%20up%20this%20device%20to%20a%20machine%20that%20can%20withstand%20the%20deep%20depths%20of%20the%20ocean,%20we%20can%20now%20explore%20parts%20of%20the%20ocean%20that%20we%20may%20not%20have%20been%20able%20to%20do%20before. HTTP/1.1" 200 -


...

currentWords: ['This', 'technology', 'could', 'used', 'night', 'vision']
Start: This
le debut 0 1
0
not find:  technology
le debut 5 1
5
not find:  could
le debut 16 1
16
not find:  used
le debut 25 1
25
not find:  night
le debut 34 1
34
not find:  vision
le debut 40 1
40
{'This': 1, 'technology': 1, 'could': 1, 'used': 1, 'night': 1, 'vision': 1}
currentWords: ['Could', 'used', 'Space', 'Station', 'detect', 'analyze', 'oncoming', 'objects']
Start: Could
le debut 0 1
0
not find:  used
le debut 9 1
9
not find:  Space
le debut 21 1
21
not find:  Station
le debut 27 1
27
not find:  detect
le debut 38 1
38
not find:  analyze
le debut 49 1
49


127.0.0.1 - - [17/Jan/2020 10:04:57] "GET /annotApi/Could%20be%20used%20by%20the%20Space%20Station%20to%20detect%20and%20analyze%20oncoming%20objects. HTTP/1.1" 200 -


not find:  oncoming
le debut 57 1
57
not find:  objects
le debut 66 1
66
{'Could': 1, 'used': 1, 'Space': 1, 'Station': 1, 'detect': 1, 'analyze': 1, 'oncoming': 1, 'objects': 1}
currentWords: ['An', 'application', 'monitors', 'suspicious', 'activity', 'outside', 'business', 'Many', 'atrocities', 'could', 'avoided', 'person', 'device', 'hidden', 'store', 'front', 'This', 'could', 'useful', 'first', 'detecting', 'threats', 'second', 'artificial', 'intelligence', 'lock', 'doors', 'weapon', 'threatening', 'person', 'approaching']
Start: An
le debut 0 1
0
not find:  application
le debut 3 1
3
not find:  monitors
le debut 20 1
20
not find:  suspicious
le debut 29 1
29
not find:  activity
le debut 40 1
40
not find:  outside
le debut 49 1
49
not find:  business
le debut 59 1
59
not find:  Many
le debut 70 1
70
not find:  atrocities
le debut 75 1
75
not find:  could
le debut 86 1
86
not find:  avoided
le debut 95 1
95
not find:  person
le debut 108 1
108
not find:  device
le debut 124 1
124
no

127.0.0.1 - - [17/Jan/2020 10:47:42] "GET /annotApi/An%20application%20that%20monitors%20suspicious%20activity%20outside%20a%20business.%20%20Many%20atrocities%20could%20be%20avoided%20if%20a%20person%20had%20this%20device%20hidden%20in%20their%20store%20front.%20%20This%20could%20be%20useful%20in%20first%20detecting%20threats,%20and%20second%20artificial%20intelligence%20which%20can%20lock%20all%20doors%20if%20a%20weapon%20or%20threatening%20person%20is%20approaching. HTTP/1.1" 200 -


ich bin da find ist true: person person
le debut 108 2
297
not find:  approaching
le debut 307 1
307
{'An': 1, 'application': 1, 'monitors': 1, 'suspicious': 1, 'activity': 1, 'outside': 1, 'business': 1, 'Many': 1, 'atrocities': 1, 'could': 2, 'avoided': 1, 'person': 2, 'device': 1, 'hidden': 1, 'store': 1, 'front': 1, 'This': 1, 'useful': 1, 'first': 1, 'detecting': 1, 'threats': 1, 'second': 1, 'artificial': 1, 'intelligence': 1, 'lock': 1, 'doors': 1, 'weapon': 1, 'threatening': 1, 'approaching': 1}
